In [1]:
import random as rd
import pandas as pd
import math
import numpy as np

from itertools import chain, combinations

In [2]:
masterTable = pd.read_csv('data.csv')

In [3]:
masterTable.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type35,Soil_Type36,Soil_Type37,Soil_Type38,Soil_Type39,Soil_Type40,Cover_Type,Unknown 1,Unknown 2,Unknown 3
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [4]:
dataset = masterTable.iloc[: , :10]

In [5]:
dataset.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points
0,2596,51,3,258,0,510,221,232,148,6279
1,2590,56,2,212,-6,390,220,235,151,6225
2,2804,139,9,268,65,3180,234,238,135,6121
3,2785,155,18,242,118,3090,238,238,122,6211
4,2595,45,2,153,-1,391,220,234,150,6172


In [6]:
def generate_subsets(arr):
    all_subsets = []
    for r in range(1, len(arr) + 1):
        subsets_r = list(combinations(arr, r))
        all_subsets.extend(subsets_r)

    return [list(subset) for subset in all_subsets]

In [7]:
def genr_uniform_distr(col,count):
  l_uni = []
  col = col.to_list()
  min_val = min(col)
  max_val = max(col)
  for i in range(count):
    l_uni.append(rd.randint(min_val,max_val))
  return l_uni

def genr_data_distr(col,count):
  l_data = []
  col =  col.to_list()
  for i in range(count):
    index = rd.choice(col)
    l_data.append(index)
  return l_data


In [8]:
def uni_widths(col,query_centre):
  col = col.to_list()
  min_val = min(col)
  max_val = max(col)
  lower_bound = 0
  if(abs(query_centre - min_val) < abs(query_centre - max_val) ):
    upper_bound = abs(query_centre - min_val)
  else:
    upper_bound = abs(query_centre - max_val)
  #print(min_val,max_val,upper_bound)
  width = np.random.uniform(0, upper_bound, 1)
  return math.ceil(width[0])

In [9]:
def expo_width(col,query_centre):
  col = col.to_list()
  min_val = min(col)
  max_val = max(col)
  lower_bound = 0
  if(abs(query_centre - min_val) < abs(query_centre - max_val) ):
    upper_bound = abs(query_centre - min_val)
  else:
    upper_bound = abs(query_centre - max_val)
  #print(min_val,max_val,upper_bound)
  scale = (upper_bound) // 4
  width = np.random.exponential(scale)
  return math.ceil(width)

In [10]:
def find_bounds(col,n):
  l_bounds = []
  if(n%2 ==0):
    n_uniform = n_data = n//2
  else:
    n_uniform = n//2
    n_data = n_uniform + 1
  #col = col.to_list()
  uniform_list = genr_uniform_distr(col,n_uniform)
  data_list = genr_uniform_distr(col,n_data)
  for i in range(n_uniform):
    query_centre = uniform_list[i]
    width = uni_widths(col,query_centre)
    lb = query_centre - width
    ub = query_centre + width
    bounds = [lb,ub]
    l_bounds.append(bounds)
  for i in range(n_data):
    query_centre = data_list[i]
    width = uni_widths(col,query_centre)
    lb = query_centre - width
    ub = query_centre + width
    bounds = [lb,ub]
    l_bounds.append(bounds)
  return l_bounds

In [11]:
def getRanges(cols, noSamplesSet):
  attToDist = dict()
  for c in cols:
    attToDist[c] = find_bounds(dataset[c], noSamplesSet)
  return attToDist

In [12]:
def create_zero_dataframe(n, D):
    data = {f"Attribute_{i}": [0.1] * n for i in range(1, D+1)}
    df = pd.DataFrame(data)
    return df

def generateSkeletonData(n, columns):
  zero_df = create_zero_dataframe(n, 2*len(columns))
  newColNames =[]
  for c in columns:
    newColNames.append(c + " LB")
    newColNames.append(c + " UB")
  zero_df.columns = newColNames
  return zero_df

In [13]:
def findIndex(col, columns):
  for i,c in enumerate(columns):
    if (c == col):
      return i

In [14]:
n = 20000
columns = list(dataset.columns)
subsets = generate_subsets(columns)

sampleSize = math.ceil(n/len(subsets))
distributions = getRanges(columns, sampleSize)
data = generateSkeletonData(sampleSize*len(subsets), columns)
ctr = 0
for s in subsets:
  for c in s:
    index = findIndex(c, columns)
    data.iloc[ctr : ctr+sampleSize, 2*index : 2*index+2] = distributions[c]
  ctr = ctr + sampleSize


In [16]:
data.shape

(20460, 20)